# Add Erroneous Data

Adding erroneous data to shrunk file for testing.

In [ ]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
!cp testing_SQ00014613.sqlite testing_err_SQ00014613.sqlite

In [ ]:
# create connections for sqlite
# reference: https://nih.figshare.com/articles/dataset/Cell_Health_-_Cell_Painting_Single_Cell_Profiles/9995672
sqlite_err_conn = create_engine("sqlite:///testing_err_SQ00014613.sqlite").connect()
sqlite_orig_conn = create_engine("sqlite:///SQ00014613.sqlite").connect()

In [ ]:
sql_stmt = """
select * from cytoplasm
"""
test_data = pd.read_sql(sql=sql_stmt, con=sqlite_err_conn)
test_data

In [ ]:
sql_stmt = """
select * from cytoplasm where Cytoplasm_Correlation_Costes_AGP_DNA = 'nan' limit 1
"""
err_data = pd.read_sql(sql=sql_stmt, con=sqlite_orig_conn)
err_data

In [ ]:
err_data["Cytoplasm_Correlation_Costes_AGP_DNA"]

In [ ]:
nan_data = err_data.apply(lambda x: "nan", axis=0).copy()
nan_data = pd.DataFrame(nan_data).T

In [ ]:
full_set = pd.concat([nan_data, err_data]).reset_index(drop=True)
full_set

In [ ]:
err_data.to_sql(name="cytoplasm", con=sqlite_err_conn, if_exists="replace", index=False)

In [ ]:
pd.read_sql(sql=sql_stmt, con=sqlite_err_conn)

In [ ]:
# close connections
sqlite_err_conn.close()
sqlite_orig_conn.close()

In [ ]:
# show relative filesizes
!ls -lh ./ | grep sqlite